In [144]:
from deep_translator import DeeplTranslator, MyMemoryTranslator, GoogleTranslator, MicrosoftTranslator
import pandas as pd
import re
import regex
import time
import random

In [61]:
from huggingface_hub import login
import importlib
import token_login
importlib.reload(token_login)
login(token= token_login.token)

In [62]:
from datasets import load_dataset

ds = load_dataset("msarmi9/korean-english-multitarget-ted-talks-task")

In [63]:
ds

DatasetDict({
    train: Dataset({
        features: ['korean', 'english'],
        num_rows: 166215
    })
    validation: Dataset({
        features: ['korean', 'english'],
        num_rows: 1958
    })
    test: Dataset({
        features: ['korean', 'english'],
        num_rows: 1982
    })
})

In [64]:
# If 'dataset' is a DatasetDict, select a split first
if isinstance(ds, dict): # Check if it's a DatasetDict
    train_df = ds['train'].to_pandas() # Example: convert the 'train' split
    test_df = ds['test'].to_pandas()
    val_df = ds['validation'].to_pandas()
else: # If 'dataset' is already a Dataset object
    df = ds.to_pandas()

In [65]:
test_df.head()

,korean,english
0,아직 3분 시작된 건 아니죠? 그렇죠?,"Allison Hunt: My three minutes hasn't started yet, has it?"
1,"크리스 앤더슨:네, 맘대로 시작하실 수 없습니다.","Chris Anderson: No, you can't start the three minutes."
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair."
3,"앨리슨 헌트 : 어머나, 여기 참 냉정하네요","AH: Oh my God, it's harsh up here."
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.


In [66]:
total_characters = sum(len(s) for s in test_df['korean'])
print(total_characters)

91236


In [11]:
langs_list = GoogleTranslator().get_supported_languages()  # output: [arabic, french, english etc...]
print(langs_list)

['afrikaans', 'albanian', 'amharic', 'arabic', 'armenian', 'assamese', 'aymara', 'azerbaijani', 'bambara', 'basque', 'belarusian', 'bengali', 'bhojpuri', 'bosnian', 'bulgarian', 'catalan', 'cebuano', 'chichewa', 'chinese (simplified)', 'chinese (traditional)', 'corsican', 'croatian', 'czech', 'danish', 'dhivehi', 'dogri', 'dutch', 'english', 'esperanto', 'estonian', 'ewe', 'filipino', 'finnish', 'french', 'frisian', 'galician', 'georgian', 'german', 'greek', 'guarani', 'gujarati', 'haitian creole', 'hausa', 'hawaiian', 'hebrew', 'hindi', 'hmong', 'hungarian', 'icelandic', 'igbo', 'ilocano', 'indonesian', 'irish', 'italian', 'japanese', 'javanese', 'kannada', 'kazakh', 'khmer', 'kinyarwanda', 'konkani', 'korean', 'krio', 'kurdish (kurmanji)', 'kurdish (sorani)', 'kyrgyz', 'lao', 'latin', 'latvian', 'lingala', 'lithuanian', 'luganda', 'luxembourgish', 'macedonian', 'maithili', 'malagasy', 'malay', 'malayalam', 'maltese', 'maori', 'marathi', 'meiteilon (manipuri)', 'mizo', 'mongolian', 'm

In [173]:
# remove speaker identification (e.g. "Bill Lange: ...")
# remove aural indicators
#r'[\uAC00-\uD7A3]+\s?[\uAC00-\uD7A3]+:\s', r'\([\uAC00-\uD7A3]+\)\s?'
pattern = [r'\w+(\s\w+)?\s?:\s?', r'\(\w+\)\s?', r'[\\p{L}]+(\s[\\p{L}]+)?\s?:\s?', r'\([\\p{L}]+\)\s?']
remove_str = "|".join(pattern)

train_df['clean english'] = train_df['english'].str.replace(remove_str, "",regex=True)
train_df['clean korean'] = train_df['korean'].str.replace(remove_str, "", regex=True)

test_df['clean english'] = test_df['english'].str.replace(remove_str, "",regex=True)
test_df['clean korean'] = test_df['korean'].str.replace(remove_str, "", regex=True)

val_df['clean english'] = val_df['english'].str.replace(remove_str, "",regex=True)
val_df['clean korean'] = val_df['korean'].str.replace(remove_str, "", regex=True)

In [175]:
test_df.head()

,korean,english,clean english,clean korean
0,아직 3분 시작된 건 아니죠? 그렇죠?,"Allison Hunt: My three minutes hasn't started yet, has it?","My three minutes hasn't started yet, has it?",아직 3분 시작된 건 아니죠? 그렇죠?
1,"크리스 앤더슨:네, 맘대로 시작하실 수 없습니다.","Chris Anderson: No, you can't start the three minutes.","No, you can't start the three minutes.","네, 맘대로 시작하실 수 없습니다."
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Reset the three minutes, that's just not fair.","3분 다시 설정해주세요, 이건 반칙입니다."
3,"앨리슨 헌트 : 어머나, 여기 참 냉정하네요","AH: Oh my God, it's harsh up here.","Oh my God, it's harsh up here.","어머나, 여기 참 냉정하네요"
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I mean I'm nervous enough as it is.,정말이지 긴장되네요


In [176]:
test_df.tail()

,korean,english,clean english,clean korean
1977,W: 예. 에너지와 관련한 일할 생각을 갖고 있습니다.,WK: Yeah. I'm still thinking to work on energy.,Yeah. I'm still thinking to work on energy.,예. 에너지와 관련한 일할 생각을 갖고 있습니다.
1978,"C: 윌리엄, TED에 당신을 초대하게 되어 영광입니다.","CA: Wow. William, it's a real honor to have you at the TED conference.","Wow. William, it's a real honor to have you at the TED conference.","윌리엄, TED에 당신을 초대하게 되어 영광입니다."
1979,와줘서 정말 고마워요.,Thank you so much for coming.,Thank you so much for coming.,와줘서 정말 고마워요.
1980,W: 감사합니다.,WK: Thank you.,Thank you.,감사합니다.
1981,(박수),(Applause),,


In [177]:
# remove empty rows
clean_train_df = train_df[train_df['clean english']!= '']
clean_test_df = test_df[test_df['clean english']!= '']
clean_val_df = val_df[val_df['clean english']!= '']
print(len(val_df))
print(len(clean_val_df))
clean_val_df.tail()

1958
1957


,korean,english,clean english,clean korean
1952,"저희 회사는 사진 7억장 보유하고 있지만,",My company has 70 million images.,My company has 70 million images.,"저희 회사는 사진 7억장 보유하고 있지만,"
1953,제 사무실에는 한 장의 사진이 있습니다.,I have one image in my office.,I have one image in my office.,제 사무실에는 한 장의 사진이 있습니다.
1954,이 사진이죠.,Here it is.,Here it is.,이 사진이죠.
1955,전 다음번에는 여러분이 여러분을 행동하게 할 사진을 만나길 바랍니다. 사진으로 이유들을 잘 이해하게 될 것입니다. 저는 강연 중에도 여러분들이 행동 할 것이라고 확신합니다.,"I hope that the next time you see an image that sparks something in you, you'll better understand why, and I know that speaking to this audience, you'll definitely do something about it.","I hope that the next time you see an image that sparks something in you, you'll better understand why, and I know that speaking to this audience, you'll definitely do something about it.",전 다음번에는 여러분이 여러분을 행동하게 할 사진을 만나길 바랍니다. 사진으로 이유들을 잘 이해하게 될 것입니다. 저는 강연 중에도 여러분들이 행동 할 것이라고 확신합니다.
1956,모든 사진가들에게 감사를 드립니다.,And thank you to all the photographers.,And thank you to all the photographers.,모든 사진가들에게 감사를 드립니다.


In [178]:
# Convert the Korean column to a list
korean_texts = clean_test_df['clean korean'].tolist()

In [220]:
result_df = pd.DataFrame(columns=['korean', 'actual translation', 'Google translation'])

# Batch size for GoogleTranslator (max = 40)
BATCH_SIZE = 30

translator = GoogleTranslator(source='ko', target='en')

google_translations = []

# Process in batches
for i in range(0, len(korean_texts), BATCH_SIZE):
    batch = korean_texts[i:i + BATCH_SIZE]
    print("batch: ", i)

    for attempt in range(3):
        try:
            translated_batch = translator.translate_batch(batch)
            google_translations.extend(translated_batch)
            break
        except Exception as e:
            print(f"Batch failed (attempt {attempt+1}): {e}")
            sleep_time = 2 ** attempt + random.random()  # exponential backoff
            print(f"Sleeping {sleep_time:.2f} sec...")
            time.sleep(sleep_time)

    time.sleep(0.3) 

# DeepL translation will stay empty unless you use DeepL Pro
# result_df['DeepL translation'] = ""

# for txt in range(0, len(clean_train_df['clean korean'])):
#     translated = GoogleTranslator(source='auto', target='en').translate(text=clean_train_df['clean korean'][txt]) 
    
#     new_row_df = pd.DataFrame({'korean': [clean_train_df['clean korean'][txt]], 'actual translation':[clean_train_df['clean english'][txt]], 'Google translation':[translated]})
#     result_df = pd.concat([result_df, new_row_df], ignore_index=True)

batch:  0
batch:  30
batch:  60
batch:  90
batch:  120
Batch failed (attempt 1): ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Sleeping 1.75 sec...
batch:  150
batch:  180
batch:  210
batch:  240
batch:  270
batch:  300
batch:  330
batch:  360
batch:  390
batch:  420
batch:  450
batch:  480
batch:  510
batch:  540
Batch failed (attempt 1): ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Sleeping 1.74 sec...
batch:  570
Batch failed (attempt 1): 아까 레모네이드 이야기가 첫번째 예입니다. --> No translation was found using the current translator. Try another translator?
Sleeping 1.89 sec...
batch:  600
batch:  630
batch:  660
batch:  690
batch:  720
batch:  750
batch:  780
batch:  810
batch:  840
batch:  870
batch:  900
batch:  930
batch:  960
batch:  990
batch:  1020
batch:  1050
batch:  1080
batch:  1110
batch:  1140
batch:  11

In [221]:
# Assemble the final DataFrame
result_df['korean'] = clean_test_df['clean korean']
result_df['actual translation'] = clean_test_df['clean english']
result_df['Google translation'] = google_translations

In [ ]:
# result_df.to_csv('data/korean_translation.csv', index=False)


In [222]:
pd.set_option('display.max_colwidth', None)

result_df.head()

,korean,actual translation,Google translation
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","The 3 minutes haven't started yet, right? Right?"
1,"네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Yes, you can't start it however you want."
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the time to 3 minutes, this is a foul."
3,"어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","Oh my, it's so cold in here."
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous.


In [223]:
langs_list = MicrosoftTranslator(api_key= token_login.microsoft_key, source='ko', target='en').get_supported_languages()
print("Supported languages (list):", langs_list)

Supported languages (list): ['afrikaans', 'amharic', 'arabic', 'assamese', 'azerbaijani', 'bashkir', 'belarusian', 'bulgarian', 'bhojpuri', 'bangla', 'tibetan', 'bodo', 'bosnian', 'catalan', 'czech', 'welsh', 'danish', 'german', 'dogri', 'lower sorbian', 'divehi', 'greek', 'english', 'spanish', 'estonian', 'basque', 'persian', 'finnish', 'filipino', 'fijian', 'faroese', 'french', 'french (canada)', 'irish', 'galician', 'konkani', 'gujarati', 'hausa', 'hebrew', 'hindi', 'chhattisgarhi', 'croatian', 'upper sorbian', 'haitian creole', 'hungarian', 'armenian', 'indonesian', 'igbo', 'inuinnaqtun', 'icelandic', 'italian', 'inuktitut', 'inuktitut (latin)', 'japanese', 'georgian', 'kazakh', 'khmer', 'kurdish (northern)', 'kannada', 'korean', 'kashmiri', 'kurdish (central)', 'kyrgyz', 'luxembourgish', 'lingala', 'lao', 'lithuanian', 'ganda', 'latvian', 'chinese (literary)', 'maithili', 'malagasy', 'māori', 'macedonian', 'malayalam', 'mongolian (cyrillic)', 'mongolian (traditional)', 'manipuri',

In [224]:
# Microsoft Translator
micro_translator = MicrosoftTranslator(api_key=token_login.microsoft_key, 
                                       endpoint = token_login.microsoft_endpoint,
                                       region= token_login.region, 
                                       source= 'ko', target='en')

In [42]:
micro_translator.translate(clean_test_df['clean korean'][0])


"It's not three minutes yet, right? Right?"

In [225]:
# Batch size for Microsoft (max = 50,000 character per request; 2mill per month)
BATCH_SIZE = 30000

microsoft_translations = []

# Process in batches
for i in range(0, len(korean_texts), BATCH_SIZE):
    batch = korean_texts[i:i + BATCH_SIZE]
    print("batch: ", i)

    for attempt in range(3):
        try:
            microsoft_translated_batch = micro_translator.translate_batch(batch)
            microsoft_translations.extend(microsoft_translated_batch)
            break
        except Exception as e:
            print(f"Batch failed (attempt {attempt+1}): {e}")
            sleep_time = 2 ** attempt + random.random()  # exponential backoff
            print(f"Sleeping {sleep_time:.2f} sec...")
            time.sleep(sleep_time)

    time.sleep(0.3) 

batch:  0


Batch failed (attempt 1): 'NoneType' object has no attribute 'json'
Sleeping 1.47 sec...


Batch failed (attempt 2): 'NoneType' object has no attribute 'json'
Sleeping 2.51 sec...


In [226]:
# Assemble the final DataFrame
microsoft_result_df = pd.DataFrame(columns=['korean', 'actual translation', 'Microsoft translation'])
microsoft_result_df['korean'] = clean_test_df['clean korean']
microsoft_result_df['actual translation'] = clean_test_df['clean english']
microsoft_result_df['Microsoft translation'] = microsoft_translations

In [227]:
pd.set_option('display.max_colwidth', None)

microsoft_result_df.tail()

,korean,actual translation,Microsoft translation
1976,"아직 19살에 불과하지만, 당신이 생각하는 좀 더 미래의 당신은… 당신은 계속 에너지와 관련된 일을 하고 있을까요?","And as you think of your life going forward, you're 19 now, do you picture continuing with this dream of working in energy?","You're only 19 years old, but what you think of as a more future you... Will you continue to work with energy?"
1977,예. 에너지와 관련한 일할 생각을 갖고 있습니다.,Yeah. I'm still thinking to work on energy.,"Yes, I have an idea of working in energy."
1978,"윌리엄, TED에 당신을 초대하게 되어 영광입니다.","Wow. William, it's a real honor to have you at the TED conference.","William, it's an honor to invite you to TED."
1979,와줘서 정말 고마워요.,Thank you so much for coming.,Thank you so much for coming.
1980,감사합니다.,Thank you.,I appreciate it.


In [ ]:
# Add Microsoft results to data
# df = pd.read_csv('data/korean_translation.csv')


In [ ]:
df = pd.merge(df, microsoft_result_df, on=['korean', 'actual translation'], how='left')
df.head()

,korean,actual translation,Google translation,Microsoft translation
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","The 3 minutes haven't started yet, right? Right?","It's not three minutes yet, right? Right?"
1,"크리스 앤더슨:네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Chris Anderson: Yes, you can't just start.","Chris Anderson: Yes, you can't start as you want."
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the time to 3 minutes, this is a foul.","Please set it again for 3 minutes, this is a foul."
3,"앨리슨 헌트 : 어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","Allison Hunt: Oh my, it's so cold in here.","Allison Hunt: Oh my God, it's so cold here"
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous.,I'm really nervous


In [ ]:
# df.to_csv('data/korean_translation.csv', index=False)

In [80]:
import deepl
deepl_translator = deepl.Translator(token_login.deepl_key)

In [ ]:
deepl_translations = []
no_translation = pd.DataFrame(columns=['i', 'non-translated text'])

In [207]:
for i in range(1900, len(korean_texts)):
    try:
        if korean_texts[i] != "":
            translated = deepl_translator.translate_text(korean_texts[i], source_lang="KO", target_lang="EN-US")
            deepl_translations.extend([translated.text])
        else:
            deepl_translations.extend([''])
    except Exception as e:
        nt = pd.DataFrame({'i':i, 'non-translated text': korean_texts[i]})
        no_translation = pd.concat([no_translation, nt], ignore_index=True)
        print("Error: ", e)
print("No Translations: ", len(no_translation))

No Translations:  0


In [208]:
len(deepl_translations)

1981

In [214]:
# Assemble the final DataFrame
deepl_result_df = pd.DataFrame(columns=['korean', 'actual translation', 'DeepL translation'])
deepl_result_df['korean'] = clean_test_df['clean korean']
deepl_result_df['actual translation'] = clean_test_df['clean english']
deepl_result_df['DeepL translation'] = deepl_translations

In [215]:
deepl_result_df

,korean,actual translation,DeepL translation
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","It's not quite three minutes in yet, is it?"
1,"네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Yes, you can't just start at will."
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the 3 minutes, this is cheating."
3,"어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","My goodness, this place is cold."
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous
...,...,...,...
1976,"아직 19살에 불과하지만, 당신이 생각하는 좀 더 미래의 당신은… 당신은 계속 에너지와 관련된 일을 하고 있을까요?","And as you think of your life going forward, you're 19 now, do you picture continuing with this dream of working in energy?","You're only 19 years old, but do you think the future you... do you think you'll still be working with energy?"
1977,예. 에너지와 관련한 일할 생각을 갖고 있습니다.,Yeah. I'm still thinking to work on energy.,Example. I'm thinking about working in energy.
1978,"윌리엄, TED에 당신을 초대하게 되어 영광입니다.","Wow. William, it's a real honor to have you at the TED conference.","William, it's an honor to have you at TED."
1979,와줘서 정말 고마워요.,Thank you so much for coming.,Thank you so much for coming.


In [229]:
df = pd.merge(result_df, microsoft_result_df, on=['korean', 'actual translation'], how='left')
df = pd.merge(df, deepl_result_df, on=['korean', 'actual translation'], how='left')
df.head()

,korean,actual translation,Google translation,Microsoft translation,DeepL translation
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","The 3 minutes haven't started yet, right? Right?","It's not three minutes yet, right? Right?","It's not quite three minutes in yet, is it?"
1,"네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Yes, you can't start it however you want.","Yes, you can't start as you want.","Yes, you can't just start at will."
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the time to 3 minutes, this is a foul.","Please set it again for 3 minutes, this is a foul.","Please reset the 3 minutes, this is cheating."
3,"어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","Oh my, it's so cold in here.","Oh my God, it's so cold here","My goodness, this place is cold."
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous.,I'm really nervous,I'm really nervous


In [230]:
df.to_csv('data/korean_translation2.csv', index=False)